# <center>XGBoost hyperparameter tuning</center>

# 1. Choosing and importing right libraries
[Optuna](https://optuna.org/) is one of the best and easiest to use libraries for optimization. Its' API allows for great flexibility when it comes to creating, stopping and editing trials making it a very important tool for Kaggle contests. Since Optuna has a specialized API for XGBoost models, we will use it to speed up the search.

In [1]:
!pip install optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 3.4 MB/s eta 0:00:00


In [2]:
import warnings
import importlib  

warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import optuna
opt_int = importlib.import_module("optuna_integration")

import pickle

import numpy as np

In [3]:
RAND = 42

In [4]:
import shutil
src_path = r"/kaggle/input/ps4e8-xgbclassifier-tuner/xgb.db"
dst_path = r"/kaggle/working/xgb.db"

try:
    shutil.copy(src_path, dst_path)
    print('Copied')
except FileNotFoundError:
    print("Not found")

Not found


# 2. Preparing data for XGBoost model
We will use XGBoost native API as it allows for better flexibility with Callbacks and early stopping which are crucial for the development. In order to do that, we need to prepare our data and turn it into DMatrix with categorical features.

In [5]:
X_train = pd.read_parquet('/kaggle/input/ps4e8-data-eng/train.parquet', engine='pyarrow')
y_train = X_train.pop('class')
dtrain = xgb.DMatrix(X_train, y_train, enable_categorical=True)

X_test = pd.read_parquet('/kaggle/input/ps4e8-data-eng/test.parquet', engine='pyarrow')
y_test = X_test.pop('class')
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

X_val = pd.read_parquet('/kaggle/input/ps4e8-data-eng/val.parquet', engine='pyarrow')
dval = xgb.DMatrix(X_val, enable_categorical=True)
assert 'class' not in X_val.columns

X_train

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
2413129,16.160,0,6,1,0,4,-1,0,16.890,17.660,3,0,0,0,1,1,5,-1,0,2
1910266,3.420,4,-1,10,0,5,2,5,3.620,15.310,-1,3,8,-1,-1,0,0,-1,0,2
1213509,8.110,0,1,8,0,-1,0,0,11.250,15.030,-1,-1,0,0,1,1,5,-1,0,2
387249,6.670,5,10,7,1,1,0,6,7.320,16.530,-1,-1,0,-1,-1,0,0,-1,0,0
2498558,12.230,0,2,5,1,4,-1,0,10.180,16.800,-1,-1,0,-1,-1,1,-1,-1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258055,3.840,1,1,5,0,0,0,1,4.820,6.100,-1,0,2,-1,-1,1,1,-1,0,2
1115921,6.290,1,2,5,0,4,0,0,5.640,9.880,-1,0,2,-1,-1,0,0,-1,0,0
2414138,8.620,0,0,4,0,4,0,6,5.360,12.680,-1,-1,0,-1,1,1,2,-1,5,0


In [6]:
train_cols = X_train.columns
train_cols

Index(['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [7]:
X_train.shape

(2493556, 20)

In [8]:
X_train.dtypes

cap-diameter            float64
cap-shape                 int64
cap-surface               int64
cap-color                 int64
does-bruise-or-bleed      int64
gill-attachment           int64
gill-spacing              int64
gill-color                int64
stem-height             float64
stem-width              float64
stem-root                 int64
stem-surface              int64
stem-color                int64
veil-type                 int64
veil-color                int64
has-ring                  int64
ring-type                 int64
spore-print-color         int64
habitat                   int64
season                    int64
dtype: object

In [9]:
def get_pred(X):
    return np.round(model.predict(X))

# 3. Defining custom XGBoost metric
Since the contest uses a custom (not available in XGBoost) metric, we need to create it ourselves! The metric used is [Matthews correlation coefficient](https://en.wikipedia.org/wiki/Phi_coefficient) known in statistics as phi coefficient. We can speed up sklearn's MCC implementation to make our search more efficient (credits go to this [notebook](https://www.kaggle.com/competitions/playground-series-s4e8/discussion/528193)).

In [10]:
def matthews_corrcoef_fast(y_true, y_pred):
    df = pd.DataFrame({'actual': y_true, 'pred': y_pred})

    # 0=p(poison), 1=e(edible)
    n11f = ((df['actual']==0) & (df['pred']==0)).sum()
    n00f = ((df['actual']==1) & (df['pred']==1)).sum()
    n10f = ((df['actual']==0) & (df['pred']==1)).sum()
    n01f = ((df['actual']==1) & (df['pred']==0)).sum()
    n1_f = n11f+n10f
    n0_f = n01f+n00f
    n_0f = n10f+n00f
    n_1f = n11f+n01f
    n1_fs = np.sqrt(n11f+n10f)
    n0_fs = np.sqrt(n01f+n00f)
    n_0fs = np.sqrt(n10f+n00f)
    n_1fs = np.sqrt(n11f+n01f)
    mcc_fast = (n11f*n00f - n10f*n01f) / (n1_fs*n0_fs*n_0fs*n_1fs)
    if np.isnan(mcc_fast): return 0
    return mcc_fast

In [12]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eta': 0.01,
        'max_depth': 64,
        'min_child_weight': 4,
        'colsample_bytree': 0.5332244728838978,
        'gamma': 0.7053913533793799,
        'alpha': trial.suggest_float('alpha', 1e-7, 0.5, log=True),
        'lambda': 0.009610147650215044,
        'seed': RAND
    }
    
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=99999999,
        evals=[(dtest, 'test')],
        verbose_eval=50,
        custom_metric=matthews_corrcoef_fast,
        callbacks=[
            xgb.callback.EarlyStopping(500, save_best=True, min_delta=1e-4, metric_name='mcc', data_name='test', maximize=True),
            opt_int.XGBoostPruningCallback(trial, 'test-mcc')
        ]
    )
    
    return matthews_corrcoef_fast(y_test, np.round(model.predict(dtest)))

# 4. Using existing Optuna study
We can reuse already existing study if it exists. This way we don't have to worry about Kaggle's 12 hour limit as we can just run this notebook multiple times each time reusing sampler and pruner from previous one. [Learn more](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/001_rdb.html).

In [13]:
study_name = 'xgb-tune'
storage = 'sqlite:///xgb.db'

In [14]:
try:
    sampler = pickle.load(open("/kaggle/input/ps4e8-xgbclassifier-tuner/sampler.pkl", "rb"))
    print("Choosing saved sampler!")
except FileNotFoundError:
    sampler = None
    print("Default sampler")

Default sampler


In [15]:
try:
    pruner = pickle.load(open("/kaggle/input/ps4e8-xgbclassifier-tuner/pruner.pkl", "rb"))
    print("Choosing saved pruner!")
except FileNotFoundError:
    pruner = optuna.pruners.HyperbandPruner()
    print("Default pruner")

Default pruner


In [16]:
study = optuna.create_study(study_name=study_name, storage=storage, direction='maximize', load_if_exists=True, sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=100)

[I 2024-08-19 17:32:54,055] A new study created in RDB with name: xgb-tune


[0]	test-logloss:0.67938	test-mcc:0.00000


/tmp/ipykernel_18/2873463128.py:17: RuntimeWarning: invalid value encountered in scalar divide
  mcc_fast = (n11f*n00f - n10f*n01f) / (n1_fs*n0_fs*n_0fs*n_1fs)


[50]	test-logloss:0.37919	test-mcc:0.98392
[100]	test-logloss:0.23053	test-mcc:0.98475
[150]	test-logloss:0.14884	test-mcc:0.98479
[200]	test-logloss:0.10267	test-mcc:0.98488
[250]	test-logloss:0.07496	test-mcc:0.98493
[300]	test-logloss:0.05874	test-mcc:0.98495
[350]	test-logloss:0.04894	test-mcc:0.98503
[400]	test-logloss:0.04314	test-mcc:0.98508
[450]	test-logloss:0.03978	test-mcc:0.98509
[500]	test-logloss:0.03780	test-mcc:0.98516
[550]	test-logloss:0.03667	test-mcc:0.98512
[600]	test-logloss:0.03606	test-mcc:0.98515
[650]	test-logloss:0.03572	test-mcc:0.98517
[700]	test-logloss:0.03554	test-mcc:0.98517
[750]	test-logloss:0.03546	test-mcc:0.98513
[800]	test-logloss:0.03544	test-mcc:0.98512
[850]	test-logloss:0.03546	test-mcc:0.98511
[900]	test-logloss:0.03548	test-mcc:0.98512
[950]	test-logloss:0.03551	test-mcc:0.98510
[982]	test-logloss:0.03552	test-mcc:0.98511


[I 2024-08-19 17:44:50,350] Trial 0 finished with value: 0.9851434734696954 and parameters: {'alpha': 2.282955238173052e-06}. Best is trial 0 with value: 0.9851434734696954.


...

/tmp/ipykernel_18/2873463128.py:17: RuntimeWarning: invalid value encountered in scalar divide
  mcc_fast = (n11f*n00f - n10f*n01f) / (n1_fs*n0_fs*n_0fs*n_1fs)
[I 2024-08-19 20:54:24,537] Trial 98 pruned. Trial was pruned at iteration 9.


[0]	test-logloss:0.67938	test-mcc:0.00000


/tmp/ipykernel_18/2873463128.py:17: RuntimeWarning: invalid value encountered in scalar divide
  mcc_fast = (n11f*n00f - n10f*n01f) / (n1_fs*n0_fs*n_0fs*n_1fs)


[50]	test-logloss:0.37907	test-mcc:0.98393
[100]	test-logloss:0.23024	test-mcc:0.98474
[150]	test-logloss:0.14869	test-mcc:0.98477
[200]	test-logloss:0.10258	test-mcc:0.98491


[I 2024-08-19 20:57:08,803] Trial 99 pruned. Trial was pruned at iteration 243.


# 5. Saving sampler and pruner
In order to be able to easily pick up current study next time, we have to save our sampler and pruner. [Learn more](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/001_rdb.html#resume-study).

In [17]:
with open("sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)
    print("Sampler saved!")
    
with open("pruner.pkl", "wb") as fout:
    pickle.dump(study.pruner, fout)
    print("Pruner saved!")

Sampler saved!
Pruner saved!


In [20]:
print(f"Best score: {study.best_value}")
print(study.best_params)

Best score: 0.9851985037906217
{'alpha': 0.06488320357425029}
